<a href="https://colab.research.google.com/github/harishk30/RingGalaxiesCNNAnalysis/blob/main/DataDownload/SEDDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np

In [ ]:
from astropy.table import Table
data = Table.read('/content/drive/MyDrive/PS1DR1_BroadMorph_clean.csv', format='csv')

In [ ]:
import os
objectIds = []
objectRas = []
objectDecs = []
objectIndices = []
for subdir, dirs, files in os.walk('/content/drive/MyDrive/ringfoldnew'):
    for file in files:
      s = file.replace('.jpg', '')
      objectIndices.append(s)
      objectIds.append(data[int(s)]['objid'])
      objectRas.append(data[int(s)]['ra'])
      objectDecs.append(data[int(s)]['dec'])

In [ ]:
print(objectIndices)

In [ ]:
rings = []
for i in range(len(data)):
  rings.append(0)
for i in range(len(objectIndices)):
  rings[int(objectIndices[i])] = 1

In [ ]:
data.add_column(rings, name = 'RING')

In [ ]:
import numpy as np
def jansky_to_ab(jansky_fluxes, zero_point):
    return -5./2. * np.log10(jansky_fluxes / zero_point)

In [ ]:
try:  # python 3
    from io import BytesIO
    from http.client import HTTPConnection
except ImportError:  # python 2
    from StringIO import StringIO as BytesIO
    from httplib import HTTPConnection

from astropy.table import Table


def query_sed(pos, radius=5):
    """ Query VizieR Photometry 
    The VizieR photometry tool extracts photometry points around a given position 
    or object name from photometry-enabled catalogs in VizieR.
    
    The VizieR photometry tool is developed by Anne-Camille Simon and Thomas Boch
    .. url:: http://vizier.u-strasbg.fr/vizier/sed/doc/
    
    Parameters
    ----------
    pos: tuple or str
        position tuple or object name
    radius: float
        position matching in arseconds.
    
    Returns
    -------
    table: astropy.Table
        VO table returned by the Vizier service.
        
    >>> query_sed((1.286804, 67.840))
    >>> query_sed("HD1")
    """
    try:
        ra, dec = pos
        target = "{0:f},{1:f}".format(ra, dec)
    except:
        target = pos
    
    url = "http:///viz-bin/sed?-c={target:s}&-c.rs={radius:f}"
    host = "vizier.u-strasbg.fr"
    port = 80
    path = "/viz-bin/sed?-c={target:s}&-c.rs={radius:f}".format(target=target, radius=radius)
    connection = HTTPConnection(host, port)
    connection.request("GET", path)
    response = connection.getresponse()
   
    table = Table.read(BytesIO(response.read()), format="votable")
    return table

In [ ]:
galex_fuv_flux = []
galex_fuv_error = []
galex_nuv_flux = []
galex_nuv_error = []
pan_g = []
pan_g_error = []
pan_r = []
pan_r_error = []
pan_i = []
pan_i_error = []
pan_z = []
pan_z_error = []
pan_y = []
pan_y_error = []
wise_w1 = []
wise_w1_error = []
wise_w2 = []
wise_w2_error = []
wise_w3 = []
wise_w3_error = []
wise_w4 = []
wise_w4_error = []
current = 0
for i in range(0, 10000):
    try:
        s = query_sed((t['RA'][i], t['DEC'][i]))
        df = s.to_pandas()
        pr = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:r']
        #Sort by magnitude if downloading Spirals
        if(jansky_to_ab(df._get_value(pr[0], 'sed_flux'), 3631) > 16.5 and jansky_to_ab(df._get_value(pr[0], 'sed_flux'), 3631) < 18.5):
          current += 1
          print(current)
          try:
            fuv = df.index.values[df['sed_filter'] == 'GALEX:FUV']
            galex_fuv_flux.append(df._get_value(fuv[0], 'sed_flux'))
            galex_fuv_error.append(df._get_value(fuv[0], 'sed_eflux'))
          except:
            galex_fuv_flux.append(-1)
            galex_fuv_error.append(-1)

          try:
            nuv = df.index.values[df['sed_filter'] == 'GALEX:NUV']
            galex_nuv_flux.append(df._get_value(nuv[0], 'sed_flux'))
            galex_nuv_error.append(df._get_value(nuv[0], 'sed_eflux'))
          except:
            galex_nuv_flux.append(-1)
            galex_nuv_error.append(-1)


          pg = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:g']
          pan_g.append(df._get_value(pg[0], 'sed_flux'))
          pan_g_error.append(df._get_value(pg[0], 'sed_eflux'))

          pr = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:r']
          pan_r.append(df._get_value(pr[0], 'sed_flux'))
          pan_r_error.append(df._get_value(pr[0], 'sed_eflux'))

          pi = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:i']
          pan_i.append(df._get_value(pi[0], 'sed_flux'))
          pan_i_error.append(df._get_value(pi[0], 'sed_eflux'))

          pz = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:z']
          pan_z.append(df._get_value(pz[0], 'sed_flux'))
          pan_z_error.append(df._get_value(pz[0], 'sed_eflux'))

          py = df.index.values[df['sed_filter'] == 'PAN-STARRS/PS1:y']
          pan_y.append(df._get_value(py[0], 'sed_flux'))
          pan_y_error.append(df._get_value(py[0], 'sed_eflux'))

          try:
            w1 = df.index.values[df['sed_filter'] == 'WISE:W1']
            wise_w1.append(df._get_value(w1[0], 'sed_flux'))
            wise_w1_error.append(df._get_value(w1[0], 'sed_eflux'))
          except:
            wise_w1.append(-1)
            wise_w1_error.append(-1)

          try:
            w2 = df.index.values[df['sed_filter'] == 'WISE:W2']
            wise_w2.append(df._get_value(w2[0], 'sed_flux'))
            wise_w2_error.append(df._get_value(w2[0], 'sed_eflux'))
          except:
            wise_w2.append(-1)
            wise_w2_error.append(-1)

          try:
            w3 = df.index.values[df['sed_filter'] == 'WISE:W3']
            wise_w3.append(df._get_value(w3[0], 'sed_flux'))
            wise_w3_error.append(df._get_value(w3[0], 'sed_eflux'))
          except:
            wise_w3.append(-1)
            wise_w3_error.append(-1)

          try:
            w4 = df.index.values[df['sed_filter'] == 'WISE:W4']
            wise_w4.append(df._get_value(w4[0], 'sed_flux'))
            wise_w4_error.append(df._get_value(w4[0], 'sed_eflux'))
          except:
            wise_w4.append(-1)
            wise_w4_error.append(-1)   
          
    except:
        galex_fuv_flux.append(-1)
        galex_fuv_error.append(-1)
        galex_nuv_flux.append(-1)
        galex_nuv_error.append(-1)
        pan_g.append(-1)
        pan_g_error.append(-1)
        pan_r.append(-1)
        pan_r_error.append(-1)
        pan_i.append(-1)
        pan_i_error.append(-1)
        pan_z.append(-1)
        pan_z_error.append(-1)
        pan_y.append(-1)
        pan_y_error.append(-1)
        wise_w1.append(-1)
        wise_w1_error.append(-1)
        wise_w2.append(-1)
        wise_w2_error.append(-1)
        wise_w3.append(-1)
        wise_w3_error.append(-1)
        wise_w4.append(-1)
        wise_w4_error.append(-1)
    print("i = " + str(i))

In [ ]:
t = Table()
t['galex_fuv_flux'] = galex_fuv_flux
t['galex_fuv_error'] = galex_fuv_error
t['galex_nuv_flux'] = galex_nuv_flux
t['galex_nuv_error'] = galex_nuv_error
t['pan_g'] = pan_g
t['pan_g_error'] = pan_g_error
t['pan_r'] = pan_r
t['pan_r_error'] = pan_r_error
t['pan_i'] = pan_i
t['pan_i_error'] = pan_i_error
t['pan_z'] = pan_z
t['pan_z_error'] = pan_z_error
t['pan_y'] = pan_y
t['pan_y_error'] = pan_y_error
t['wise_w1'] = wise_w1
t['wise_w1_error'] = wise_w1_error
t['wise_w2'] = wise_w2
t['wise_w2_error'] = wise_w2_error
t['wise_w3'] = wise_w3
t['wise_w3_error'] = wise_w3_error
t['wise_w4'] = wise_w4
t['wise_w4_error'] = wise_w4_error

In [ ]:
print(t)

    galex_fuv_flux        galex_fuv_error     ...     wise_w4_error    
---------------------- ---------------------- ... ---------------------
 4.320000152802095e-05  6.799999937356915e-06 ... 0.0012000000569969416
                  -1.0                   -1.0 ... 0.0014100000262260437
                  -1.0                   -1.0 ...                   nan
1.7500000467407517e-05  3.900000137946336e-06 ... 0.0008200000156648457
                  -1.0                   -1.0 ...                   nan
                  -1.0                   -1.0 ...                   nan
                  -1.0                   -1.0 ...  0.001019999966956675
                  -1.0                   -1.0 ...                   nan
1.6199999663513154e-05 3.5000000480067683e-06 ...   0.00139999995008111
                  -1.0                   -1.0 ...                   nan
                   ...                    ... ...                   ...
2.7399999453336932e-05  3.900000137946336e-06 ... 0.001099999994

In [ ]:
df = t.to_pandas()
df.to_csv('/content/drive/MyDrive/fluxring.csv')